# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-17 03:52:17] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=38438, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1047832347, constrained_json_whitespace_pattern=None, w

[2025-06-17 03:52:17] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-17 03:52:28] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 03:52:28] Init torch distributed begin.


[2025-06-17 03:52:29] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 03:52:29] Load weight begin. avail mem=63.55 GB


...

[2025-06-17 03:52:34] Init torch distributed begin.
[2025-06-17 03:52:34] Init torch distributed ends. mem usage=0.00 GB
[2025-06-17 03:52:34] Load weight begin. avail mem=23.13 GB
[2025-06-17 03:52:34] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]

[2025-06-17 03:52:35] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=19.31 GB, mem usage=3.82 GB.
[2025-06-17 03:52:35] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-17 03:52:35] Memory pool end. avail mem=19.00 GB


[2025-06-17 03:52:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=19.13 GB
[2025-06-17 03:52:36] INFO:     Started server process [117277]
[2025-06-17 03:52:36] INFO:     Waiting for application startup.
[2025-06-17 03:52:36] INFO:     Application startup complete.
[2025-06-17 03:52:36] INFO:     Uvicorn running on http://127.0.0.1:38438 (Press CTRL+C to quit)


[2025-06-17 03:52:36] INFO:     127.0.0.1:56990 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 03:52:37] INFO:     127.0.0.1:56992 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 03:52:37] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-17 03:53:42] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-17 03:54:12] INFO:     127.0.0.1:57006 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 03:54:12] The server is fired up and ready to roll!


[2025-06-17 03:54:15] INFO:     127.0.0.1:60142 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-17 03:54:15] Child process unexpectedly failed with exitcode=9. pid=117981


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-17 03:54:24] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=38050, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1054969687, constrained_json_whitespace_pattern=None, wat

[2025-06-17 03:54:24] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-17 03:54:32] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 03:54:32] Init torch distributed begin.


[2025-06-17 03:54:33] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 03:54:33] Load weight begin. avail mem=62.21 GB
[2025-06-17 03:54:33] Using model weights format ['*.safetensors']


...

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]

[2025-06-17 03:54:38] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.82 GB, mem usage=0.93 GB.
[2025-06-17 03:54:38] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-17 03:54:38] Memory pool end. avail mem=37.51 GB


[2025-06-17 03:54:39] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.69 GB
[2025-06-17 03:54:39] INFO:     Started server process [125355]
[2025-06-17 03:54:39] INFO:     Waiting for application startup.
[2025-06-17 03:54:39] INFO:     Application startup complete.
[2025-06-17 03:54:39] INFO:     Uvicorn running on http://127.0.0.1:38050 (Press CTRL+C to quit)


[2025-06-17 03:54:40] INFO:     127.0.0.1:52450 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-17 03:54:40] INFO:     127.0.0.1:52458 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 03:54:40] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:54:44] INFO:     127.0.0.1:52470 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 03:54:44] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-17 03:54:45] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:54:46] INFO:     127.0.0.1:52472 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-17 03:54:46] Child process unexpectedly failed with exitcode=9. pid=125937


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-17 03:54:53] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=37364, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=255386318, constrained_json_whitespa

[2025-06-17 03:55:01] Casting torch.bfloat16 to torch.float16.


[2025-06-17 03:55:02] Casting torch.bfloat16 to torch.float16.
[2025-06-17 03:55:02] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 03:55:02] Init torch distributed begin.


[2025-06-17 03:55:02] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 03:55:03] Load weight begin. avail mem=61.36 GB


...

[2025-06-17 03:55:20] INFO:     Started server process [127675]
[2025-06-17 03:55:20] INFO:     Waiting for application startup.
[2025-06-17 03:55:20] INFO:     Application startup complete.
[2025-06-17 03:55:20] INFO:     Uvicorn running on http://127.0.0.1:37364 (Press CTRL+C to quit)


[2025-06-17 03:55:20] INFO:     127.0.0.1:43852 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 03:55:21] INFO:     127.0.0.1:43854 - "GET /get_model_info HTTP/1.1" 200 OK


[2025-06-17 03:55:21] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:55:23] INFO:     127.0.0.1:43862 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 03:55:23] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-17 03:55:25] Detected chat template content format: string
[2025-06-17 03:55:25] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:55:25] INFO:     127.0.0.1:43866 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-17 03:55:25] Child process unexpectedly failed with exitcode=9. pid=128258


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-17 03:55:33] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=34500, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1032288967, constrained_json_whitespace_patte

[2025-06-17 03:55:33] Casting torch.bfloat16 to torch.float16.


[2025-06-17 03:55:40] Casting torch.bfloat16 to torch.float16.


[2025-06-17 03:55:40] Casting torch.bfloat16 to torch.float16.
[2025-06-17 03:55:40] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 03:55:40] Init torch distributed begin.


[2025-06-17 03:55:41] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-17 03:55:58] INFO:     Started server process [129803]
[2025-06-17 03:55:58] INFO:     Waiting for application startup.
[2025-06-17 03:55:58] INFO:     Application startup complete.
[2025-06-17 03:55:58] INFO:     Uvicorn running on http://127.0.0.1:34500 (Press CTRL+C to quit)


[2025-06-17 03:55:58] INFO:     127.0.0.1:34024 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 03:55:59] INFO:     127.0.0.1:34028 - "GET /get_model_info HTTP/1.1" 200 OK


[2025-06-17 03:55:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:56:02] INFO:     127.0.0.1:34042 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 03:56:02] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-17 03:56:03] Detected chat template content format: string
[2025-06-17 03:56:03] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:56:04] INFO:     127.0.0.1:51876 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-17 03:56:04] Child process unexpectedly failed with exitcode=9. pid=130015
[2025-06-17 03:56:04] Child process unexpectedly failed with exitcode=9. pid=129949


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-17 03:56:11] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=37275, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=407879461, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_time

[2025-06-17 03:56:18] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 03:56:18] Init torch distributed begin.


[2025-06-17 03:56:18] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 03:56:19] Load weight begin. avail mem=78.50 GB


[2025-06-17 03:56:19] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-06-17 03:56:24] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-17 03:56:25] INFO:     Started server process [130820]
[2025-06-17 03:56:25] INFO:     Waiting for application startup.
[2025-06-17 03:56:25] INFO:     Application startup complete.
[2025-06-17 03:56:25] INFO:     Uvicorn running on http://0.0.0.0:37275 (Press CTRL+C to quit)


[2025-06-17 03:56:26] INFO:     127.0.0.1:47756 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 03:56:26] INFO:     127.0.0.1:47764 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 03:56:26] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:56:27] INFO:     127.0.0.1:47770 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 03:56:27] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-17 03:56:31] Detected chat template content format: string
[2025-06-17 03:56:31] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 03:56:31] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 10.30, #queue-req: 0


[2025-06-17 03:56:32] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 115.19, #queue-req: 0


[2025-06-17 03:56:33] Decode batch. #running-req: 1, #token: 229, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 107.85, #queue-req: 0


[2025-06-17 03:56:33] Decode batch. #running-req: 1, #token: 298, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 107.01, #queue-req: 0


...

[2025-06-17 03:56:35] Decode batch. #running-req: 1, #token: 500, token usage: 0.02, accept len: 1.62, cuda graph: False, gen throughput (token/s): 103.29, #queue-req: 0


[2025-06-17 03:56:36] Decode batch. #running-req: 1, #token: 565, token usage: 0.03, accept len: 1.62, cuda graph: False, gen throughput (token/s): 102.66, #queue-req: 0


[2025-06-17 03:56:36] Decode batch. #running-req: 1, #token: 634, token usage: 0.03, accept len: 1.73, cuda graph: False, gen throughput (token/s): 109.67, #queue-req: 0


[2025-06-17 03:56:37] Decode batch. #running-req: 1, #token: 699, token usage: 0.03, accept len: 1.62, cuda graph: False, gen throughput (token/s): 104.81, #queue-req: 0


[2025-06-17 03:56:37] INFO:     127.0.0.1:53968 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).